In [1]:
# General imports
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import normalized_mutual_info_score
from scipy.spatial import distance, distance_matrix
import matplotlib.pyplot as plt

#### Import Data

In [9]:

# KDD Dataset
# First column is the BLOCK ID (class label), BLOCK IDs are integers running from 1 to 303 with 153 unique values (k)
# Second column is the ELEMENT ID (sample number), unique numbers, not ordered
# Third column is the class of the example. Homologous proteins = 1, non-homologous proteins = 0
data = pd.read_csv('bio_train.csv',skiprows=0).to_numpy(dtype='object')


#####################
# Toy Dataset
#dataset = np.genfromtxt('dataset1_noCluster7.csv', delimiter = ',')[1:]
#dataset_ft = dataset[:,:2]
#dataset_lb = dataset[:,-1]

#scaler = StandardScaler()
#dataset_ft = scaler.fit_transform(dataset_ft)

#plt.scatter(dataset_ft[:,0], dataset_ft[:,1], c = dataset_lb)
#plt.show()

(145749, 77)


'\nTemporary dataset for testing purposes. 2 features, 7 clusters\n'

#### Data Preprocessing

In [10]:
# Shuffle, split into labels/features and normalize data
def process_data(data):
    # Shuffle
    shuffle = np.random.permutation(len(data))
    data = data[shuffle]
    
    # Split
    block_ids = data[:,0]
    element_ids = data[:,1]
    homology = data[:,2]
    features = data[:,3:]
    
    # Normalize the features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    
    return block_ids, element_ids, features

In [11]:
# preprocess the data
block_ids, element_ids, features = process_data(data)

### Task 1 - Lloyds Algorithm

In [14]:
def lloyds(data, k=153):
    # Number of samples and features of the dataset
    
    n_samples, n_features = np.shape(data)
    
    # Pick k random points from data to be the initial cluster centers (eventually use kmeans+ here?)
    rand_nums = np.random.randint(0,n_samples,k)
    cluster_means = data[rand_nums]
    
    old_means = np.zeros([k, n_features])
    counter = 0
    
    while (old_means != cluster_means).any():
    
        counter += 1
        print("iteration: ",counter)
        old_means = np.copy(cluster_means)
        
        # measure assingment runtime
        start_assign = time.perf_counter()
        
        ############# Assign step
        # first approach
        dist_matrix = distance_matrix(data, cluster_means)
        cluster_labels = np.argmin(dist_matrix, axis=1)
        
        
        # second approach for computing the distances
        #cluster_labels = np.zeros(n_samples)
        #sample_dist = np.zeros(k)
        #for i in range(n):
        #    for j in range(k):
        #        sample_dist[j] = distance.sqeuclidean(data[i], cluster_means[j])
        #    cluster_labels[i] = np.argmin(sample_dist)
        
        # measure update runtime
        start_update = time.perf_counter()
        
        ############# Update step
        for j in range(k):
            point_sum = np.zeros(n_features)
            point_nr = 0
            for i in range(n_samples):
                if cluster_labels[i] == j:
                    point_nr += 1
                    point_sum += data[i]
            
            if point_nr == 0:
                cluster_means[j] = np.copy(old_means[j])
            else:
                cluster_means[j] = point_sum/point_nr
            
        end_update = time.perf_counter()
        
        print('Assign step runtime: '+str(start_update - start_assign))
        print('Update step  runtime: '+str(end_update - start_update))
        
    print('KMeans converged in '+str(counter)+' iterations.')
    return cluster_labels, cluster_means

In [15]:
labels, centers = lloyds(features)
#labels, centers = lloyds(dataset_ft, 7)

iteration:  1
Assign step runtime: 11.218267659000048
Update step  runtime: 4.288877942999989
iteration:  2
Assign step runtime: 11.547677550999992
Update step  runtime: 4.280310646000089
iteration:  3
Assign step runtime: 11.186793286000011
Update step  runtime: 4.115879010999947
iteration:  4
Assign step runtime: 11.068957857999976
Update step  runtime: 4.2998429700001
iteration:  5
Assign step runtime: 12.28592508600002
Update step  runtime: 4.481656117000057
iteration:  6
Assign step runtime: 11.803110372999981
Update step  runtime: 4.224168856999995
iteration:  7
Assign step runtime: 12.193432981
Update step  runtime: 4.249796253999989
iteration:  8
Assign step runtime: 11.72605226099995
Update step  runtime: 4.247098960000017
iteration:  9
Assign step runtime: 11.651492904999941
Update step  runtime: 4.233769107000057
iteration:  10
Assign step runtime: 11.651313229000039
Update step  runtime: 4.302390816999946
iteration:  11
Assign step runtime: 11.801234926000006
Update step  r

KeyboardInterrupt: 

In [ ]:
#plt.scatter(dataset_ft[:,0], dataset_ft[:,1], c = labels)
#plt.scatter(centers[:,0], centers[:,1], c='r')
#plt.show()

In [7]:
NMI_score = normalized_mutual_info_score(block_ids, labels)
print(NMI_score)

0.0


### Task 2 - LSH + Kmeans

In [133]:
# this function defines a hash function according to the notes on LSH + Kmeans and assigns 
# the samples to the buckets.
# There is still a mistake in this function

def hash_simple(data, no_buckets):
    
    no_samples = len(data)
    
    hash_values = np.zeros(no_samples)
    
    vector_p = np.random.normal(loc=0.0, scale=1.0, size=len(data[0]))
    
    for i in range(n):
        hash_values[i] = data[i].dot(vector_p)
        
    min_val = np.min(hash_values)
    max_val = np.max(hash_values)
    
    bucket_size = (max_val-min_val) / (no_buckets-1)
    
    return np.floor(hash_values/bucket_size)